In [1]:
import pandas as pd
from transformers import AutoTokenizer
from src.utils.score_utils import metrics_mapping
import torch

2023-02-06 05:41:26.708999: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-06 05:41:26.709028: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
devcands = pd.read_csv("/mnt/data1/prasann/latticegen/lattice-generation/COMET/data/PARPREFIX_DEV.csv")

In [3]:
devcands = devcands.rename(columns={"mt":"hyp"})

In [4]:
len(devcands)

66814

In [5]:
tmpcands = devcands[devcands['plen']<13]

In [28]:
with torch.no_grad():
    metrics_mapping("pqe", tmpcands)

81.40172364641126 %
82.8052016403149 %
84.20867963421856 %
85.61215762812219 %
87.01563562202583 %
88.41911361592948 %
89.82259160983311 %
91.22606960373676 %
92.62954759764041 %
94.03302559154405 %
95.43650358544768 %
96.83998157935133 %
98.24345957325498 %
99.64693756715862 %
TOOK TIME  1841.08


/mnt/data1/prasann/tfr-decoding/src/utils/score_utils.py:313: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tset[metric] = get_scores_auto(hyps, srcs, refs, "parentqe", "comstyle")


In [2]:
tmpcands = pd.read_csv("prefix_scored.csv")

In [6]:
#allcands = pd.read_csv("tmp.csv", index_col=0)
def rer_met(rer, tgt, df, preflen):
    N=1
    tmp = df[df['plen']==preflen]
    print(tmp[tgt].mean())
    fsort = tmp.sort_values(by=['ref', rer], ascending=[True, False]).groupby('ref', as_index=False).nth[:N]
    return fsort[tgt].mean()

In [18]:
print(rer_met("prefpqe", "score", tmpcands, 9))
print(rer_met("pqe", "score", tmpcands, 9))

0.6357526586236814
0.6622992265826895
0.6357526586236814
0.6771156381147742


In [19]:
#testcands = list(allcands['hyp'][:50])
#testscos = list(allcands['precision'][:50])
tok = AutoTokenizer.from_pretrained("facebook/bart-base")

In [13]:
allcands = allcands.drop(columns=['precision', 'recall', 'f1'])
allcands = allcands.loc[allcands.hyp.apply(lambda x: isinstance(x, str))]
allcands = allcands.dropna()

In [ ]:
metrics_mapping("parent", allcands)

In [18]:
allcands.to_csv("updated_dset_full.csv")

In [4]:
# explore how well scores of prefixes predicts which choice will lead to something good down the line
# TODO use Jiacheng's code for beam search latticification?
# TODO use the T5 t2t generation model

In [19]:
def extract_prefixes(cands, tok):
    return None
    
def tok_str(toks):
    s = ""
    for t in toks:
        s = s+str(t)+"_"
    return s[:-1]

def get_prefs_info(pint, tokd, scos):
    infos = {}
    for t in range(len(tokd)):
        if len(tokd[t])<pint:
            continue
        k = tok_str(tokd[t][:pint]) # get prefix key
        # for each prefix, track inds of cands w that prefix
        if k in infos:
            infos[k].append(scos[t])
        else:
            infos[k] = [scos[t]]
    
    return infos

def get_prefix_data(tokd, scos, plen, tok):
    pinfo = get_prefs_info(plen, tokd, scos)
    result = []
    for inf in pinfo.keys():
        # get usable prefix string
        pref = tok.decode([int(f) for f in inf.split("_")])
        # each prefix associated with best possibility
        result.append({
            "inp":pref,
            "scos":pinfo[inf],
            "sco":max(pinfo[inf]),
            "plen":plen
        })
    return result

# given selected part of df, get prefix data
def ex_dset(exs, plen, tok):
    tokd = tok(list(exs['hyp']), add_special_tokens=False).input_ids
    maxl = max(len(t) for t in tokd)
    pind = plen
    pdata = []
    while pind<maxl:
        pdata.extend(get_prefix_data(tokd, list(exs['precision']), pind, tok))
        pind+=plen
    result = pd.DataFrame(pdata)
    result['src'] = exs['src'].iloc[0]
    result['ref'] = exs['ref'].iloc[0]
    return result
    
def get_full_dset(acands, plen, tok):
    alldfs = []
    uns = acands['ref'].unique()
    ind = 0
    lens = []
    print(len(uns))
    for u in uns:
        try:
            alldfs.append(ex_dset(acands[acands['ref']==u], plen, tok))
            ind+=1
        except:
            print("anomaly")
        if ind%1000==0:
            print(ind)
    return pd.concat(alldfs), [len(adf) for adf in alldfs]

# TODO remember to remove add_special_tokens when tokenizing
    

In [20]:
fullset, lens = get_full_dset(allcands, 3, tok)

16242
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000


In [21]:
#fullset.to_csv("prefix_dset.csv")

In [6]:
fullset = pd.read_csv("prefix_dset.csv")

In [7]:
fullset = fullset.rename(columns={'sco':'score', "inp":"mt"})

In [8]:
split = int(len(fullset)*.8)
devsize = int(len(fullset)*.95)

trainset = fullset.iloc[:split].sample(frac=1).reset_index()
devset = fullset.iloc[devsize:].sample(frac=1).reset_index()

In [9]:
trainset.to_csv("/mnt/data1/prasann/latticegen/lattice-generation/COMET/data/PARPREFIX_TRAIN.csv")
devset.to_csv("/mnt/data1/prasann/latticegen/lattice-generation/COMET/data/PARPREFIX_DEV.csv")

In [56]:
infos = get_prefs_info(10, tokd, testscos)

In [57]:
infos

{'0_287_1843_8_2708_20484_459_9429_446_34_': [0.7275104059794265,
  0.8043923419964522,
  0.7501542733805745,
  0.7022985883539701,
  1.0,
  0.8043923419964522,
  0.791947919692525,
  0.6355653131400284,
  1.0,
  0.8857000285382948,
  0.6652049901111008,
  0.8881501211769413],
 '0_20_34177_12270_11_5_496_10315_9_15541_': [1.0,
  0.874235093122898,
  0.874235093122898,
  0.7949253342322761,
  0.874235093122898],
 '0_20_34177_12270_9_287_1843_8_2708_20484_': [0.8560118665583999,
  0.8153343052686353,
  0.8153551038173115,
  0.7328616209964707],
 '0_287_1843_8_2708_20484_459_9429_446_16_': [0.7679249640195699,
  0.6975863464034741,
  0.7679249640195699,
  0.8012120435729666,
  0.781285189930025,
  0.7679249640195699,
  0.7679249640195699,
  0.825189076232105,
  0.8043923419964522,
  0.8209592487442797,
  0.7306804610333764,
  0.7679249640195699,
  0.7080321137411774,
  0.6975863464034741,
  0.7679249640195699],
 '0_20_34177_12270_13_287_1843_8_2708_20484_': [0.8811286209119195],
 '0_20_34